In [1]:
# import libraries

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow
import pickle

In [12]:
# load the trained model ,scaler pickle, one hot
model = tensorflow.keras.models.load_model('model.h5')
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)
with open('label_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)


In [13]:
input_data={
    'creditScore':600,
    'Geography':'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,    
    'EstimatedSalary': 50000
}

In [10]:
print(label_encoder_geo.classes_)


['Female' 'Male']


In [14]:
# One-hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [16]:
# combine the one-hot encoded columns with the original input data
input_df = pd.DataFrame([input_data])
input_df

,creditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [17]:

# Combine one-hot encoded 'Geography' with the rest of the input data
input_data = pd.concat([input_df.reset_index(drop=True), geo_encoded_df], axis=1)
input_data

,creditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [18]:
## Encode Categorical variables 
input_df['Gender']=label_encoder_geo.transform(input_df['Gender'])
input_df

,creditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [ ]:

## concatenation one hot encoded data
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df


,creditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [25]:
import numpy as np
input_df = np.array(input_df)


In [26]:
## Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

c:\Users\Rajeev Pandey\.virtualenvs\LoyaltyLens-YKD759Qs\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [27]:
# predcition
prediction=model.predict(input_scaled)
prediction 

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 540ms/step


array([[0.03053174]], dtype=float32)

In [28]:
prediction_proba=prediction[0][0]

In [29]:
prediction_proba

np.float32(0.030531742)

In [30]:
if prediction_proba > 0.5:
    print("The customer is likely to leave the bank.")  
else:
    print("The customer is likely to stay with the bank.")  

The customer is likely to stay with the bank.
